In [8]:
from cgeniepy.foram import ForamModel as fm
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "Fira Sans"

def biomass_diff(xmodel, xtime, ymodel=None, ytime=None, percentage=False):
    if not ymodel:
        #print("Detecting the same model")
        ymodel = xmodel

    # to avoid "if not zero = True"
    if not ytime and ytime != 0:
        #print("Detecting the same time")
        ytime = xtime

    foram_names = ["bn", "bs", "sn", "ss"]
    x_assem, y_assem = [],[]

    for foram in foram_names:
        x_state_foram = fm(xmodel).select_foramtype(foram).biomass().isel(time=xtime).sum().magnitude
        y_state_foram = fm(ymodel).select_foramtype(foram).biomass().isel(time=ytime).sum().magnitude
        x_assem.append(x_state_foram)
        y_assem.append(y_state_foram)

    # calculate squared chord distance for each model
    assemblage_1 = np.array(x_assem).sum()
    assemblage_2 = np.array(y_assem).sum()
    diff = assemblage_1 -assemblage_2
    if percentage:
        return diff/assemblage_2 * 100
    else:
        return diff

In [9]:
mod_paths = [
    "../model/worjh2.RpCO2_Rp13CO2.Albani.2100.4deg.nonspinose",
    "../model/worjh2.RpCO2_Rp13CO2.Albani.2100.4deg.asymbiosis",
    "../model/worjh2.RpCO2_Rp13CO2.Albani.2100.4deg.spinose",
    "../model/worjh2.RpCO2_Rp13CO2.Albani.2100.4deg.symbiosis",
    "../model/worjh2.RpCO2_Rp13CO2.Albani.2100.4deg",
]

fig, ax = plt.subplots(1, 1)

labels = ['Non-spinose only', 'Asymbiotic only', 'Spinose only','Symbiotic only', 'Full']

for position in ['top','bottom','left','right']:
    ax.spines[position].set_linewidth(1)
    ax.spines[position].set_color("black")

for i, ipath in enumerate(mod_paths):
    biomass = []
    time_label = fm(ipath).select_var("time").array.values - 4.5
    # print 2000 year and afterwards
    for j in range(len(time_label)):
        biomass.append(biomass_diff(xmodel=ipath, xtime=j, ytime=0, percentage=True))
    biomass = np.array(biomass)
    ax.plot(time_label, biomass, label=labels[i], linewidth=2)


ax.set_xlabel("Year")
ax.grid(True, which='both', linestyle='--', linewidth=0.3)
ax.legend()

ax.set_ylabel("Total biomass change under 4 degrees warming (%)")
fig.savefig("../output/diversity_increases_resilience.png", dpi=300)